## Librerias a utilizar

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
df = pd.read_csv('Menu_Items.csv')
names = df[' Name'].tolist()
stopwords = ['de', 'y']


In [3]:
users  = pd.read_csv('Users.csv')

In [4]:
lista = pd.read_csv('ListComidaCena.csv')

In [5]:
#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words=stopwords)

#Replace NaN with an empty string
df[' Name'] = df[' Name'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df[' Name']) 
tfidf_matrix.shape

(5640, 4096)

In [6]:
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [7]:
indices = pd.Series(df.index, index=df[' Name']).drop_duplicates()

In [8]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df[' Name'].iloc[movie_indices]

In [9]:
get_recommendations('Piña').tolist()

['Zumo de piña Hacendado',
 'Media piña',
 'Piña natural a rodajas',
 'Zumo de piña y uva Hacendado',
 'Zumo de piña y uva Hacendado',
 'Bebida de piña Hacendado sin azúcares añadidos',
 'Bebida de piña Hacendado sin azúcares añadidos',
 'Zumo de piña selección Hacendado',
 'Zumo de piña selección Hacendado',
 'Bebida láctea de piña y coco Hacendado']

## Recomendar cestas de la compra

In [10]:
df1 = df.iloc[:,:3]

In [11]:
disccionario_total = {
    'Zumos' : ['zumo', 'néctar' ,'leche','tropical','frutas', 'multivitaminas', 'smoothie', 'mango', 'pomelo', 'naranja'],
    'Cereales' : ['cereales'],
    'Frutas' : ['cerezas','albaricoque','néctar', 'manzana', 'plátano', 'fresa', 'fresas', 'coco', 'frambuesa', 'uvas', 'ciruela', 'higos', 'paraguayo', 'arándanos', 'nectarina', 'melocotón', 'papaya', 'naranja', 'mandarina', 'uva', 'peras', 'pera', 'piña', 'lima', 'pomelo', 'manzanas', 'manzana', 'banana', 'ciruelas', 'paraguayos', 'frambuesas', 'aguacates', 'mango', 'aguacate', 'mini', 'limas', 'limón', 'melón', 'sandía', 'kiwis', 'moras', 'limones' ],
    'Verduras' : ['ensalada','alcachofas','champiñones','alcachofa','verduras','verdura', 'pimientos', 'espinacas', 'cebolla','haba', 'rollitos', 'rodajas', 'lechiga', 'iceberg', 'perejil', 'zanahoria', 'brócoli', 'berenjena', 'jengibre', 'judía','berenjenas', 'verde', 'coliflor', 'pepino', 'escarola', 'alcachofa', 'apio', 'tomate', 'puerro', 'calabaza', 'calabacín', 'pimiento', 'papa', 'patata', 'remolacha', 'zanahoria', 'cebolla', 'ajo','ajos','pisto','espárragos', 'judías',  'puerro', 'rábano', 'yuca', 'maíz',  'rábano', 'nabo','ajo', 'remolacha','hierbabuena', 'espárrago', 'puerro', 'apio', 'espinaca', 'rúcula', 'lechuga', 'endivia', 'acelga', 'canónigos', 'coliflor', 'brócoli', 'alcachofa', 'puerros', 'rabanitos', 'acelgas', 'champiñón', 'seta', 'cebollino', 'hiervabuena', 'bimi', 'zanahorias', 'tomates', 'cebollas', 'batata', 'lechugas'],
    'Legumbres' : ['garbanzo','legumbre', 'garbanzos', 'lentejas', 'alubias', 'lenteja', 'alubia'],
    'Pescados' : ['berberecho','pescado', 'bacalao', 'rolls', 'mejillones','almejas', 'bonito', 'sardinillas', 'marina', 'almejones','cardo','sardinillas', 'langostino', 'huevas', 'atún', 'rodaballo', 'rape', 'merluza', 'pargo', 'trucha', 'gallo', 'dorada', 'lubina', 'corvino', 'sardina', 'boquerón', 'lenguado', 'calamar', 'anillas', 'marisco', 'anguriñas', 'rodaja', 'sepia', 'pescanova', 'camarones', 'navajas', 'carabinero', 'gamba', 'mejillon', 'alistado', 'gambón', 'caviar', 'corvina', 'chipirón', 'gallos', 'aritos', 'corvina', 'pulpo', 'rabas', 'filete', 'filetes', 'muslitos', 'mejillón', 'almeja', 'berberechos', 'nécora', 'bogavante'],
    'Carnes' : ['carne'],
    'Lacteos' : ['burrata','bífidus', 'smoothie', 'lácteo', 'natillas', 'queso', 'leche', 'helado', 'yogur', 'crema', 'mantequilla', 'danonino','petit', 'kéfir', 'danone', 'cuajada'],
    'Huevos' : ['huevo', 'tortilla', 'huevos'],
    'Grasas' : ['grasa'],
    'Bebidas' : ['cortado', 'zumo','bebida', 'exprimido', 'agua', 'coca-cola', 'fanta', 'gaseosa', 'cola', 'refresco', 'energético', 'energética', 'malta', 'cafeína', 'tónica', 'schweppes', 'kas', 'limonada', 'infusión', 'té', 'café', 'soluble', 'vino', 'lambrusco', 'cava', 'sidra', 'licor', 'vodka', 'orujo', 'ron', 'ginebra', 'mojito', 'whisky', 'brandy', 'vermouth', 'cerveza', 'champagne', 'granadina', 'moscatel', 'tinto', 'miel', 'pacharán', 'sangría'],
    'Vinagre' : ['vinagre', 'vinagreta'],
    'Sopas' : ['salmorejo','sopa', 'gazpacho'],
    'Postres' : ['tiramisú', 'postre', 'mousse', 'gelatina', 'tarta', 'pastel', 'flan', 'helado', 'galleta', 'dulce', 'chocolate', 'mermelada', 'macedonia', 'panna', 'ensaimadas', 'medias', 'berlina', 'trenza', 'napolitana'],
    'Sazonadores' : ['quinoa','ñoras','guindillas','cúrcuma','jengubre','grano','canela','comino','curry','pimientas','granulado','molida', 'pimentón','cilantro', 'pimienta', 'sazonador', 'sal', 'bicarbonato', 'orégano', 'perejil', 'hierbas', 'romero', 'laurel', 'albahaca','colorante', 'hojas', 'tomillo', 'azafrán'],
    'Proteicos' : ['pollo,','proteínas', 'pollo', 'salmón', 'desnatada', 'añadidos', 'fresco', 'avena', 'integral', 'arroz', 'cacahuete', '0%', 'integrales'],
    'Embutidos' : ['compango','york','quesos','morcilla', 'ibéricos', 'ibérico','jamón', 'chorizo', 'tocino', 'chistorra', 'salchicha', 'salami', 'embutido', 'paletilla', 'ibérica', 'salchicón', 'longaniza', 'cecina', 'jamoncitos', 'vacuno', 'paleta', 'hueso', 'mortadela', 'lonchas', 'pavo', 'fiambre', 'redondas', 'lonchas'],
    'Dulce' : ['chocolate', 'galletas', 'dulce', 'cheesecake', 'copa','caramelo', 'caramelo', 'galleta', 'tarta', 'pastel', 'brownie', 'turrón', 'pastelería', 'rosca' 'nata', 'cupcakes', 'repostería','ensaimada', 'bizcocho', 'cacao', 'choco',  'medias', 'berlina', 'trenza', 'napolitana', 'palmeritas', 'mantecadas','fartons', 'pastas', 'gofres', 'sobaos', 'croissant', 'valencianas', 'saladas', 'horchata' ],
    'Poco_habituales' : ['manzanilla', 'ponche', 'chopped','cous','tapioca','edulcorante','popitos','sur','palmito','germinados','papilla','requesón','guisantes','educolorantes','pañales', 'perro','anti-insectos','razas','perro','dental','bolsas','tortugas','roedores','alimento','spray','agar','vainilla','caracoles','vegetales','cangrejo','tsatsiki', 'oriental', 'tofu', 'hummus', 'velas', 'vela', 'merengue', 'migas', 'decoraciones', 'sirope', 'extracto', 'lecho', 'perros', 'gato', 'gatos', ],
    'Pan' : ['pan', 'tostada', 'palitos', 'picatostes', 'picos', 'piquitos', 'baguette', 'bocadillo', 'bocadillos', 'hogaza', 'regañá', 'panecillos', 'bocados', 'panes', 'piñones', 'panecillo', 'baguettes', 'panadería'],
    'Pasta' : ['hélices','medialunas','fideuá','tortiglioni','helice','tallarín','cannelloni','fusilli','pasta', 'spaghetti', 'masa', 'fideos', 'canelloni', 'canelones', 'fideo', 'penne', 'girasoles', 'macarrón', 'tallarines'],
    'frutos_secos' : ['almendras', 'almendra', 'pistacho', 'cacahuete', 'avellana', 'mix', 'altramuces', 'cocktail', 'semillas', 'nuez', 'dátiles', 'pipas', 'pasas', 'cuquis', 'anacardo', 'castaña', 'piparra', 'castañas', 'aperitivos', 'cortezas'],
    'Habituales' : ['ensaladilla', 'patatas', 'patata', 'arroz', 'tomate', 'mozzarella'],
    'Para_tener' : ['aceitunas', 'arroz', 'pasta', 'hojaldre', 'foie', 'paté', 'lacón', 'setas'],
    'Congelados' : ['ultracongelado', 'ultracongelados', 'ultracongelada', 'ultracongeladas', 'congelado', 'congelados', 'congelada', 'congeladas', 'helado', 'helados', 'hielo', 'hielos'],
    'Hogar' : ['caja','algodón','fósforos', 'funda', 'serrín', 'pila', 'mechero', 'encendedor', 'fosforos', 'velote', 'pajitas', 'bolsa', 'aluminio', 'lavavajillas', 'pastillas', 'bolsa', 'reparador', 'cloro', 'colgador', 'kit', 'ph', 'lejía', 'detergente', 'tiras', 'limpiagafas', 'quitagrasas', 'ambientador', 'eliminador', 'friegasuelos', 'colgador', 'piscina', 'tubo', 'limpiador'],
    'Cuidado_personal' : ['cuerpo','hidratante','neceser','deliplus','maquillaje', 'sombra', 'pincel', 'máscara', 'pestañas', 'perfilador', 'pintalabios', 'labial', 'brillo', 'polvo', 'polvera', 'colorete', 'corrector','betún', 'toallitas', 'polvos', 'lote', 'protector', 'aftersun', 'colonia', 'eau', 'necceser', 'pómez', 'cepillo', 'corta', 'tijera', 'lima', 'laca', 'uñas', 'gel', 'compresa', 'enjuague','decolorante', 'cabello', 'dentífrico', 'dentales', 'esponja', 'jabón', 'desodorante', 'pinza', 'sérum', 'facial', 'bálsamo', 'corporal', 'preservativos', 'mascarillas', 'lágrimas', 'picaduras', 'raíces', 'mechas', 'coloración', 'champú', 'acondicionador', 'fósforos', 'algodón', 'tampones', 'mascarilla', 'protegeslip', 'loción', 'fragancia', 'faciales', 'ojos', 'ampollas', 'espuma', 'comprimidos', 'polen'],
    'Hortalizas' : ['cebolla', 'ajo', 'espinaca', 'alcachofa', 'zanahoria', 'albahaca', 'calabaza', 'remolacha', 'pepinillos'],
    'Salsas' : ['mayonesa', 'salsa', 'ketchup', 'mostaza', 'allioli'],
    'Bebes' : ['papilla', 'infantil', 'crecimiento', 'baby', 'bebé'],
    'Desayunos' : ['cereales','aceite',  'tostadas', 'tartaletas', 'biscotes', 'galletas', 'churros', 'tortitas', 'leche', 'colacao', 'crepes', 'bizcoho', 'bollo', 'magdalenas', 'pancakes', 'empanada', 'caracola', 'margarina', 'barritas', 'muesli'],
    'Reposteria' : ['relleno','tarta', 'pastel', 'brownie', 'turrón', 'pastelería', 'rosca' 'nata', 'cupcakes', 'repostería', 'crepes', 'levadura', 'harina', 'preparado', 'bizcocho', 'hojaldre', 'bizcochos', 'tortas', ],
    'Caprichos' : ['snack','chocolate', 'pizza', 'tarta', 'hamburgesa', 'azúcar', 'mayonesa', 'nata', 'crakers', 'tortillas', 'banderillas', 'coquito', 'golosionas', 'caramelos', 'chicle', 'educolorante', 'regaliz', 'gominolas', 'paté', 'golosinas', 'juegos', 'surtido', 'chicles', 'nachos', 'pringles', 'torreznos', 'cebollitas', 'picadillo'],
    'Aceites_vinagre_sal' : ['aceite','vinagre','sal'],
    'Carnes' : ['sobrasada', 'chucrut','bacón', 'carne', 'boloñesa', 'costilla', 'costillas', 'lomo', 'cerdo', 'pollo', 'pechuga', 'hamburguesa', 'hamburguesas', 'cocido', 'albóndigas','entrecot','burger', 'fabada', 'callos', 'lasaña','bacon', 'panceta','tacos', 'brochetas', 'cordero', 'chuletas', 'conejo', 'panceta', 'vacuno', 'churrasco', 'nuggets', 'burguer', 'butifarra', 'pierna', 'solomillo', 'magret', 'codornices', 'gallina',  'salchichas', 'pollo,', 'repollo', 'pieza', 'chalotas']

    }

In [16]:
tags = {

'virgen extra':['virgen extra'],

'Hacendado': ['hacendado'] ,

'aceite coco': ['aceite de coco'] ,

'especias': ['perejil', 'laurel', 'pimienta', 'pimentón', 'curry', 'eneldo', 'cilantro', 'azafrán', 'canela', 'ajo', 'nuez moscada', 'sal de ajo', 'cúrcuma', 'cayena', 'trufas', 'ñoras', 'anís', 'pimientos choriceros', 'piñones'],
'salsa': ['salsa', 'mayonesa', 'ketchup', 'mostaza', 'allioli', 'tomate frito'],

'bebida isotónica': ['bebida isotónica'],

'refresco':[ 'refresco' ]  ,

'tónica': ['tónica' ],

'limonada':[ 'limonada'],

'aperitivos':[ 'aceitunas', 'mix de encurtidos', 'banderillas', 'jalapeños', 'alcaparrones', 'ajos aliñados', 'pepinillos', 'patatas fritas', 'pistacho', 'almendra', 'anacardo', 'papas', 'cacahuete', 'maíz', 'nuez', 'combinado frutos secos', 'cocktail', 'doritos', 'tiras de maíz', 'cuquitos', 'torreznos', 'cortezas', 'cheetos'],

'té': ['te'],
'café': ['cafe'],

'carne': ['pollo', 'ternera', 'cerdo', 'compango', 'gallina', 'cocido', 'conejo', 'cordero', 'vaca', 'hamburguesa', 'bacón' ],

'congelado': ['congelado', 'congelados'],

'sin gluten':['sin gluten' ],

'cereales': ['cereales' ],

'galletas': ['galletas'],

'chocolate': ['chocolate'],

'tortitas': ['tortitas'],

'pavo': ['pavo'] ,

'longaniza': ['longaniza'],
'pollo': ['pollo'] ,

'jamón': ['jamón' ],

'untar':[ 'paté', 'untar', 'foie', 'mantequilla', 'humus', 'sobrasada'],

'queso': ['queso' ],

'lonchas':[ 'lonchas' ],

'sin lactosa': ['sin lactosa'],

'atún': ['atún' ],

'bonito':[ 'bonito' ],
'caballa': ['caballa' ],

'mejillones': ['mejillones' ],

'escabeche':[ 'escabeche' ],

'conservas': ['espárragos', 'macedonia de verduras', 'remolacha', 'palmito', 'La Hacienda', 'apio', 'pimiento', 'cardo', 'acelgas', 'chucrut', 'brotes'],

'maíz': ['maíz'],

'gazpacho': ['gazpacho' ],

'salmorejo': ['salmorejo'],

'sopa': ['sopa' ],

'acondicionador': ['acondicionador' ],

'mascarilla': ['mascarilla' ],

'sérum': ['sérum'] ,

'champú':[ 'champú' ],

'cabello': ['champú' , 'sérum', 'mascarilla', 'acondicionador', 'coloración', 'laca', 'espuma cabello', 'gel fijador', 'cera', 'peine', 'cepillo', 'pinza cabello', 'gomas negras', 'coleteros', 'horquilla', 'cabello' ] , 

'infantil': ['kids', 'infantil' ],
'tinte': ['coloración' ],

'laca': ['laca' ],

'gel fijador': ['gel fijador'],

'cera':[ 'cera' ],

'peine': ['cepillo' ],

'afeitar': ['afeitar', 'after shave' ],

'after shave': ['after shave'],

'cuidado': ['cuidado', 'corporal', 'manos', 'toallitas', 'pinza', 'pinzas', 'tónico', 'facial', 'mascarilla', 'desodorante', 'gel'  ],

'crema manos': ['crema manos'],

'exfoliante': ['exfoliante'] ,

'mascarilla':[ 'mascarilla'],

'depilación': ['depilación', 'bandas de cera', 'depilatoria', 'cejas' ],

'dentífrico': ['dentífrico', 'caries' ],

'higiene íntima': ['compresa', 'protegeslip', 'tampones', 'pañal adulto', 'toallitas'],

'manicura': ['uñas', 'quitaesmalte', 'lima uñas', 'pómez'],

'pedicura':[ 'pómez', 'uñas', 'quitaesmalte', 'lima uñas'],

'pizza': ['pizza'],

'perfume': ['perfume', 'parfum', 'eau de toilette'],
'colonia': ['colonia', 'eau de toilette'],
'protector solar': ['protector solar', 'aceite solar'], 

'parafarmacia': ['alcohol 96º', 'agua oxigenada', 'tiras protectoras', 'yodada', 'yodo', 'esparadrapo', 'alcohol de romero', 'gasa', 'gasas'],

'limpieza hogar': ['detergente', 'agua destilada', 'lejía', 'limpiahogar', 'desinfectante', 'limpiador'],

'baño': ['baño', 'baños' ] ,

'quitamanchas': ['quitamanchas' ],

'guantes': ['guantes'],

'Ambientador': ['ambientador' ],


'maquillaje': ['maquillaje', 'sombra de ojos', 'pincel'],

'colorete': ['colorete', 'polvo suelto', 'polvo compacto'],

'preparado': ['preparado'],

'salmón': ['salmón'],

'bacalao':[ 'bacalao' ],

'calamar': ['calamar' ],

'gallo': ['gallo' ],

'lenguado': ['lenguado' ],

'corvina': ['corvina'] ,

'dorada': ['dorada'],

'lubina': ['lubina' ],

'trucha': ['trucha'] ,

'pargo': ['pargo'],

'ahumado': ['ahumado'],
'sushi':[ 'sushi', 'nigiris', 'bento', 'rolls', 'makis', 'gyozas', 'noodles' ],

'mascotas': ['gato', 'perro', 'periquitos', 'tortugas', 'cobaya', 'conejos enanos', 'mascotas' ],

'bollería': ['berlina', 'trenza', 'rosquillas', 'pastel de crema', 'napolitana', 'bizcocho', 'donut', 'muffin', 'muffins', 'magdalenas', 'bollo', 'gofres', 'panettone', 'palmera', 'polvorón', 'mazapán', 'tarta', 'macarons', 'roscón', 'merengue', 'cupcakes' ],
'empanada':[ 'empanada' ],

'harina':[ 'harina' ],

'pan': ['baguette',' pan de molde,' 'pan', 'hogaza'],

'picos': ['picos' ],

'tarta': ['tarta' ],

'roscón': ['roscón' ],

'cumpleaños': ['cumpleaños', 'vela'],

'rosca':[ 'rosca'] ,

'listo para comer': ['Listo para Comer'],

'preparado': ['Listo para Comer'],

'gazpacho': ['gazpacho' ] ,

'salmorejo': ['salmorejo' ] ,
'bifidus': ['bidifus' ],

'postres': ['gelatina', 'yogur', 'copa crema', 'tiramisú', 'cuajada', 'tocino de cielo', 'coulant', 'copa de avellana', 'mousse', 'postre'],

'proteína':[ 'proteína', 'proteico' ],

'zumo': ['zumo', 'fruta + leche', 'tropical', 'bebida tropical', 'smoothie' ],
'sin grasa': ['zero materia grasa']

}

## Recomendador con nuevas categorias

#### 2.1 Cluster “No me quito nada, como de todo” 

In [12]:
disccionario_2_1 = {'Desayuno': ['cereales','pan', 'tostada', 'palitos', 'picatostes', 'picos', 'piquitos', 'baguette', 'bocadillo', 'bocadillos', 'hogaza', 'regañá', 'panecillos', 'bocados', 'panes', 'piñones', 'panecillo', 'baguettes', 'panadería','aceite',  'tostadas', 'tartaletas', 'biscotes', 'galletas', 'churros', 'tortitas', 'leche', 'colacao', 'crepes', 'bizcoho', 'bollo', 'magdalenas', 'pancakes', 'empanada', 'caracola', 'margarina', 'barritas', 'muesli','cerezas','albaricoque','néctar', 'manzana', 'plátano', 'fresa', 'fresas', 'coco', 'frambuesa', 'uvas', 'ciruela', 'higos', 'paraguayo', 'arándanos', 'nectarina', 'melocotón', 'papaya', 'naranja', 'mandarina', 'uva', 'peras', 'pera', 'piña', 'lima', 'pomelo', 'manzanas', 'manzana', 'banana', 'ciruelas', 'paraguayos', 'frambuesas', 'aguacates', 'mango', 'aguacate', 'mini', 'limas', 'limón', 'melón', 'sandía', 'kiwis', 'moras', 'limones','burrata','bífidus', 'smoothie', 'lácteo', 'natillas', 'queso', 'leche', 'helado', 'yogur', 'crema', 'mantequilla', 'danonino','petit', 'kéfir', 'danone', 'cuajada'],
 'comida_cena':['huevo', 'tortilla', 'huevos','hélices','medialunas','fideuá','tortiglioni','helice','tallarín','cannelloni','fusilli','pasta', 'spaghetti', 'masa', 'fideos', 'canelloni', 'canelones', 'fideo', 'penne', 'girasoles', 'macarrón', 'tallarines','garbanzo','legumbre', 'garbanzos', 'lentejas', 'alubias', 'lenteja', 'alubia','sobrasada', 'chucrut','bacón', 'carne', 'boloñesa', 'costilla', 'costillas', 'lomo', 'cerdo', 'pollo', 'pechuga', 'hamburguesa', 'hamburguesas', 'cocido', 'albóndigas','entrecot','burger', 'fabada', 'callos', 'lasaña','bacon', 'panceta','tacos', 'brochetas', 'cordero', 'chuletas', 'conejo', 'panceta', 'vacuno', 'churrasco', 'nuggets', 'burguer', 'butifarra', 'pierna', 'solomillo', 'magret', 'codornices', 'gallina',  'salchichas', 'pollo,', 'repollo', 'pieza', 'chalotas' , 'berberecho','pescado', 'bacalao', 'rolls', 'mejillones','almejas', 'bonito', 'sardinillas', 'marina', 'almejones','cardo','sardinillas', 'langostino', 'huevas', 'atún', 'rodaballo', 'rape', 'merluza', 'pargo', 'trucha', 'gallo', 'dorada', 'lubina', 'corvino', 'sardina', 'boquerón', 'lenguado', 'calamar', 'anillas', 'marisco', 'anguriñas', 'rodaja', 'sepia', 'pescanova', 'camarones', 'navajas', 'carabinero', 'gamba', 'mejillon', 'alistado', 'gambón', 'caviar', 'corvina', 'chipirón', 'gallos', 'aritos', 'corvina', 'pulpo', 'rabas', 'filete', 'filetes', 'muslitos', 'mejillón', 'almeja', 'berberechos', 'nécora', 'bogavante','cebolla', 'ajo', 'espinaca', 'alcachofa', 'zanahoria', 'albahaca', 'calabaza', 'remolacha', 'pepinillos', 'grasa', 'salmorejo','sopa', 'gazpacho','ensaladilla', 'patatas', 'patata', 'arroz', 'tomate', 'mozzarella', 'pollo,','proteínas', 'pollo', 'salmón', 'desnatada', 'añadidos', 'fresco', 'avena', 'integral', 'arroz', 'cacahuete', '0%', 'integrales', 'tiramisú', 'postre', 'mousse', 'gelatina', 'tarta', 'pastel', 'flan', 'helado', 'galleta', 'dulce', 'chocolate', 'mermelada', 'macedonia', 'panna', 'ensaimadas', 'medias', 'berlina', 'trenza', 'napolitana', 'ensalada','alcachofas','champiñones','alcachofa','verduras','verdura', 'pimientos', 'espinacas', 'cebolla','haba', 'rollitos', 'rodajas', 'lechiga', 'iceberg', 'perejil', 'zanahoria', 'brócoli', 'berenjena', 'jengibre', 'judía','berenjenas', 'coliflor', 'pepino', 'escarola', 'alcachofa', 'apio', 'tomate', 'puerro', 'calabaza', 'calabacín', 'pimiento', 'papa', 'patata', 'remolacha', 'zanahoria', 'cebolla', 'ajo','ajos','pisto','espárragos', 'judías',  'puerro', 'rábano', 'yuca', 'maíz',  'rábano', 'nabo','ajo', 'remolacha','hierbabuena', 'espárrago', 'puerro', 'apio', 'espinaca', 'rúcula', 'lechuga', 'endivia', 'acelga', 'canónigos', 'coliflor', 'brócoli', 'alcachofa', 'puerros', 'rabanitos', 'acelgas', 'champiñón', 'seta', 'cebollino', 'hiervabuena', 'bimi', 'zanahorias', 'tomates', 'cebollas', 'batata', 'lechugas'],
 'entre_horas': ['chocolate', 'galletas', 'dulce', 'cheesecake', 'copa','caramelo', 'caramelo', 'galleta', 'tarta', 'pastel', 'brownie', 'turrón', 'pastelería', 'rosca' 'nata', 'cupcakes', 'repostería','ensaimada', 'bizcocho', 'cacao', 'choco',  'medias', 'berlina', 'trenza', 'napolitana', 'palmeritas', 'mantecadas','fartons', 'pastas', 'gofres', 'sobaos', 'croissant', 'valencianas', 'saladas', 'horchata', 'snack','chocolate', 'pizza', 'tarta', 'hamburgesa', 'azúcar', 'mayonesa', 'nata', 'crakers', 'tortillas', 'banderillas', 'coquito', 'golosionas', 'caramelos', 'chicle', 'educolorante', 'regaliz', 'gominolas', 'paté', 'golosinas', 'juegos', 'surtido', 'chicles', 'nachos', 'pringles', 'torreznos', 'cebollitas', 'picadillo', 'compango','york','quesos','morcilla', 'ibéricos', 'ibérico','jamón', 'chorizo', 'tocino', 'chistorra', 'salchicha', 'salami', 'embutido', 'paletilla', 'ibérica', 'salchicón', 'longaniza', 'cecina', 'jamoncitos', 'vacuno', 'paleta', 'hueso', 'mortadela', 'lonchas', 'pavo', 'fiambre', 'redondas', 'lonchas'],
 'Hogar': ['caja','algodón','fósforos', 'funda', 'serrín', 'pila', 'mechero', 'encendedor', 'fosforos', 'velote', 'pajitas', 'bolsa', 'aluminio', 'lavavajillas', 'pastillas', 'bolsa', 'reparador', 'cloro', 'colgador', 'kit', 'ph', 'lejía', 'detergente', 'tiras', 'limpiagafas', 'quitagrasas', 'ambientador', 'eliminador', 'friegasuelos', 'colgador', 'piscina', 'tubo', 'limpiador','higiénico'],
  'Cuidado_personal' : ['cuerpo','hidratante','neceser','deliplus','maquillaje', 'sombra', 'pincel', 'máscara', 'pestañas', 'perfilador', 'pintalabios', 'labial', 'brillo', 'polvo', 'polvera', 'colorete', 'corrector','betún', 'toallitas', 'polvos', 'lote', 'protector', 'aftersun', 'colonia', 'eau', 'necceser', 'pómez', 'cepillo', 'corta', 'tijera', 'lima', 'laca', 'uñas', 'gel', 'compresa', 'enjuague','decolorante', 'cabello', 'dentífrico', 'dentales', 'esponja', 'jabón', 'desodorante', 'pinza', 'sérum', 'facial', 'bálsamo', 'corporal', 'preservativos', 'mascarillas', 'lágrimas', 'picaduras', 'raíces', 'mechas', 'coloración', 'champú', 'acondicionador', 'fósforos', 'algodón', 'tampones', 'mascarilla', 'protegeslip', 'loción', 'fragancia', 'faciales', 'ojos', 'ampollas', 'espuma', 'comprimidos', 'polen']

} 


#### 2.2 “No suelo comer mucha carne”

In [13]:
diccionario_2_2 = {'Desayuno':['cereales', 'pan', 'tostada', 'palitos', 'picatostes', 'picos', 'piquitos', 'baguette', 'bocadillo', 'bocadillos', 'hogaza', 'regañá', 'panecillos', 'bocados', 'panes', 'piñones', 'panecillo', 'baguettes', 'panadería','cereales','aceite',  'tostadas', 'tartaletas', 'biscotes', 'galletas', 'churros', 'tortitas', 'leche', 'colacao', 'crepes', 'bizcocho', 'bollo', 'magdalenas', 'pancakes', 'empanada', 'caracola', 'margarina', 'barritas', 'muesli', 'cerezas','albaricoque','néctar', 'manzana', 'plátano', 'fresa', 'fresas', 'coco', 'frambuesa', 'uvas', 'ciruela', 'higos', 'paraguayo', 'arándanos', 'nectarina', 'melocotón', 'papaya', 'naranja', 'mandarina', 'uva', 'peras', 'pera', 'piña', 'lima', 'pomelo', 'manzanas', 'manzana', 'banana', 'ciruelas', 'paraguayos', 'frambuesas', 'aguacates', 'mango', 'aguacate', 'mini', 'limas', 'limón', 'melón', 'sandía', 'kiwis', 'moras', 'limones','burrata','bífidus', 'smoothie', 'lácteo', 'natillas', 'queso', 'leche', 'helado', 'yogur', 'crema', 'mantequilla', 'danonino','petit', 'kéfir', 'danone', 'cuajada'], 
'Comida_Cena' :  ['huevo', 'tortilla', 'huevos','hélices','medialunas','fideuá','tortiglioni','helice','tallarín','cannelloni','fusilli','pasta', 'spaghetti', 'masa', 'fideos', 'canelloni', 'canelones', 'fideo', 'penne', 'girasoles', 'macarrón', 'tallarines','garbanzo','legumbre', 'garbanzos', 'lentejas', 'alubias', 'lenteja', 'alubia','sobrasada', 'chucrut','bacón', 'carne', 'boloñesa', 'costilla', 'costillas', 'lomo', 'cerdo', 'pollo', 'pechuga', 'hamburguesa', 'hamburguesas', 'cocido', 'albóndigas','entrecot','burger', 'fabada', 'callos', 'lasaña','bacon', 'panceta','tacos', 'brochetas', 'cordero', 'chuletas', 'conejo', 'panceta', 'vacuno', 'churrasco', 'nuggets', 'burguer', 'butifarra', 'pierna', 'solomillo', 'magret', 'codornices', 'gallina',  'salchichas', 'pollo,', 'repollo', 'pieza', 'chalotas' , 'berberecho','pescado', 'bacalao', 'rolls', 'mejillones','almejas', 'bonito', 'sardinillas', 'marina', 'almejones','cardo','sardinillas', 'langostino', 'huevas', 'atún', 'rodaballo', 'rape', 'merluza', 'pargo', 'trucha', 'gallo', 'dorada', 'lubina', 'corvino', 'sardina', 'boquerón', 'lenguado', 'calamar', 'anillas', 'marisco', 'anguriñas', 'rodaja', 'sepia', 'pescanova', 'camarones', 'navajas', 'carabinero', 'gamba', 'mejillon', 'alistado', 'gambón', 'caviar', 'corvina', 'chipirón', 'gallos', 'aritos', 'corvina', 'pulpo', 'rabas', 'filete', 'filetes', 'muslitos', 'mejillón', 'almeja', 'berberechos', 'nécora', 'bogavante','cebolla', 'ajo', 'espinaca', 'alcachofa', 'zanahoria', 'albahaca', 'calabaza', 'remolacha', 'pepinillos', 'grasa', 'salmorejo','sopa', 'gazpacho', 'ensaladilla', 'patatas', 'patata', 'arroz', 'tomate', 'mozzarella', 'pollo,','proteínas', 'pollo', 'salmón', 'desnatada', 'añadidos', 'fresco', 'avena', 'integral', 'arroz', 'cacahuete', '0%', 'integrales', 'tiramisú', 'postre', 'mousse', 'gelatina', 'tarta', 'pastel', 'flan', 'helado', 'galleta', 'dulce', 'chocolate', 'mermelada', 'macedonia', 'panna', 'ensaimadas', 'medias', 'berlina', 'trenza', 'napolitana', 'ensalada','alcachofas','champiñones','alcachofa','verduras','verdura', 'pimientos', 'espinacas', 'cebolla','haba', 'rollitos', 'rodajas', 'lechiga', 'iceberg', 'perejil', 'zanahoria', 'brócoli', 'berenjena', 'jengibre', 'judía','berenjenas', 'coliflor', 'pepino', 'escarola', 'alcachofa', 'apio', 'tomate', 'puerro', 'calabaza', 'calabacín', 'pimiento', 'papa', 'patata', 'remolacha', 'zanahoria', 'cebolla', 'ajo','ajos','pisto','espárragos', 'judías',  'puerro', 'rábano', 'yuca', 'maíz',  'rábano', 'nabo','ajo', 'remolacha','hierbabuena', 'espárrago', 'puerro', 'apio', 'espinaca', 'rúcula', 'lechuga', 'endivia', 'acelga', 'canónigos', 'coliflor', 'brócoli', 'alcachofa', 'puerros', 'rabanitos', 'acelgas', 'champiñón', 'seta', 'cebollino', 'hiervabuena', 'bimi', 'zanahorias', 'tomates', 'cebollas', 'batata', 'lechugas'],
'Entre_horas': ['chocolate', 'galletas', 'dulce', 'cheesecake', 'copa','caramelo', 'caramelo', 'galleta', 'tarta', 'pastel', 'brownie', 'turrón', 'pastelería', 'rosca' 'nata', 'cupcakes', 'repostería','ensaimada', 'bizcocho', 'cacao', 'choco',  'medias', 'berlina', 'trenza', 'napolitana', 'palmeritas', 'mantecadas','fartons', 'pastas', 'gofres', 'sobaos', 'croissant', 'valencianas', 'saladas', 'horchata', 'snack','chocolate', 'pizza', 'tarta', 'hamburgesa', 'azúcar', 'mayonesa', 'nata', 'crakers', 'tortillas', 'banderillas', 'coquito', 'golosionas', 'caramelos', 'chicle', 'educolorante', 'regaliz', 'gominolas', 'paté', 'golosinas', 'juegos', 'surtido', 'chicles', 'nachos', 'pringles', 'torreznos', 'cebollitas', 'picadillo', 'compango','york','quesos','morcilla', 'ibéricos', 'ibérico','jamón', 'chorizo', 'tocino', 'chistorra', 'salchicha', 'salami', 'embutido', 'paletilla', 'ibérica', 'salchicón', 'longaniza', 'cecina', 'jamoncitos', 'vacuno', 'paleta', 'hueso', 'mortadela', 'lonchas', 'pavo', 'fiambre', 'redondas', 'lonchas'],
'Hogar' :['caja','algodón','fósforos', 'funda', 'serrín', 'pila', 'mechero', 'encendedor', 'fosforos', 'velote', 'pajitas', 'bolsa', 'aluminio', 'lavavajillas', 'pastillas', 'bolsa', 'reparador', 'cloro', 'colgador', 'kit', 'ph', 'lejía', 'detergente', 'tiras', 'limpiagafas', 'quitagrasas', 'ambientador', 'eliminador', 'friegasuelos', 'colgador', 'piscina', 'tubo', 'limpiador', 'higiénico'],
'Cuidado_personal' : ['cuerpo','hidratante','neceser','deliplus','maquillaje', 'sombra', 'pincel', 'máscara', 'pestañas', 'perfilador', 'pintalabios', 'labial', 'brillo', 'polvo', 'polvera', 'colorete', 'corrector','betún', 'toallitas', 'polvos', 'lote', 'protector', 'aftersun', 'colonia', 'eau', 'necceser', 'pómez', 'cepillo', 'corta', 'tijera', 'lima', 'laca', 'uñas', 'gel', 'compresa', 'enjuague','decolorante', 'cabello', 'dentífrico', 'dentales', 'esponja', 'jabón', 'desodorante', 'pinza', 'sérum', 'facial', 'bálsamo', 'corporal', 'preservativos', 'mascarillas', 'lágrimas', 'picaduras', 'raíces', 'mechas', 'coloración', 'champú', 'acondicionador', 'fósforos', 'algodón', 'tampones', 'mascarilla', 'protegeslip', 'loción', 'fragancia', 'faciales', 'ojos', 'ampollas', 'espuma', 'comprimidos', 'polen']

}

#### 2.3 “Dieta equilibrada, sin excesos”

In [17]:
# value of a list is in another lits
def is_in_list(a, b):
    a = a.split(' ')
    a = [word.lower() for word in a]
    return any(i in a for i in b)

In [31]:
df1['tag'] = ''
for row in range(len(df1)):
    for nombre, categoria in tags.items():
        if is_in_list(df1[' Name'][row], categoria):
            #add to column of df
            df1['tag'][row] += ' '+str(nombre)

c:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [32]:
# group by tag ans set name as a list of products
df11 = df1.groupby('tag')[' Name'].apply(list).reset_index(name='products')

In [25]:
#separate with comma columns values
df1['tag'] = df1['tag'].str.replace(' ', ',')

In [38]:
# drop tags that are not in tags diccitonary keys
df12 = df11[df11['tag'].isin(tags.keys())]

In [40]:
tags.keys()

dict_keys(['virgen extra', 'Hacendado', 'aceite coco', 'especias', 'salsa', 'bebida isotónica', 'refresco', 'tónica', 'limonada', 'aperitivos', 'té', 'café', 'carne', 'congelado', 'sin gluten', 'cereales', 'galletas', 'chocolate', 'tortitas', 'pavo', 'longaniza', 'pollo', 'jamón', 'untar', 'queso', 'lonchas', 'sin lactosa', 'atún', 'bonito', 'caballa', 'mejillones', 'escabeche', 'conservas', 'maíz', 'gazpacho', 'salmorejo', 'sopa', 'acondicionador', 'mascarilla', 'sérum', 'champú', 'cabello', 'infantil', 'tinte', 'laca', 'gel fijador', 'cera', 'peine', 'afeitar', 'after shave', 'cuidado', 'crema manos', 'exfoliante', 'depilación', 'dentífrico', 'higiene íntima', 'manicura', 'pedicura', 'pizza', 'perfume', 'colonia', 'protector solar', 'parafarmacia', 'limpieza hogar', 'baño', 'quitamanchas', 'guantes', 'Ambientador', 'maquillaje', 'colorete', 'preparado', 'salmón', 'bacalao', 'calamar', 'gallo', 'lenguado', 'corvina', 'dorada', 'lubina', 'trucha', 'pargo', 'ahumado', 'sushi', 'mascotas

In [42]:
df11.to_excel('tags.xlsx', index=False)

In [29]:
#list of products have each tag 
df1['tag'] = df1['tag'].str.split(',')
df1['tag'] = df1['tag'].apply(lambda x: list(set(x)))

TypeError: 'float' object is not iterable

In [30]:
df1.to_csv('df_2_2.csv', index=False)

In [160]:
#split df column values
df1['tipo'] = df1['tipo'].str.split('_')
# drop [] from tipo column

In [157]:

df1['tipo'] = df1['tipo'].apply(lambda x: x[0])

TypeError: 'float' object is not subscriptable

In [118]:
df1.to_csv('Productos_final.csv', index=False)

In [31]:
df1

,Name,Description,Price,tipo
0,Zumo de manzana selección Hacendado,"Brick 1 L | 1,40 €/L",1.40,Desayuno
1,Zumo de pomelo Hacendado,"Brick 1 L | 1,30 €/L",1.30,Desayuno
2,Bebida de pera Hacendado sin azúcares añadidos,"Brick 1 L | 1,00 €/L",1.00,Desayuno Comida_Cena
3,Bebida guayaba Hacendado sin azúcares añadidos,"Brick 1 L | 1,20 €/L",1.20,Comida_Cena
4,Bebida de pomelo Hacendado sin azúcares añadidos,"Brick 1 L | 1,10 €/L",1.10,Desayuno Comida_Cena
...,...,...,...,...
5635,Aceite de oliva virgen extra Hacendado Gran Se...,"Botella 750 ml | 6,00 €/L",6.00,Desayuno
5636,Aceite de oliva virgen extra Hacendado 1L,"Botella 1 L | 4,25 €/L",4.25,Desayuno
5637,Aceite de oliva virgen extra Hacendado 3L,"Garrafa 3 L | 4,234 €/L",12.70,Desayuno
5638,"Aceite de oliva 0,4º Hacendado 1L","Botella 1 L | 3,90 €/L",3.90,Desayuno


In [38]:
disccionario_2_1.keys()

dict_keys(['Desayuno', 'comida_cena', 'entre_horas', 'Hogar', 'Cuidado_personal'])

In [39]:
 df1[df1['tipo'].str.contains('comida_cena')]

,Name,Description,Price,tipo
2,Bebida de pera Hacendado sin azúcares añadidos,"Brick 1 L | 1,00 €/L",1.00,Desayuno comida_cena
3,Bebida guayaba Hacendado sin azúcares añadidos,"Brick 1 L | 1,20 €/L",1.20,comida_cena
4,Bebida de pomelo Hacendado sin azúcares añadidos,"Brick 1 L | 1,10 €/L",1.10,Desayuno comida_cena
5,Bebida de mango Hacendado sin azúcares añadidos,"Brick 1 L | 0,85 €/L",0.85,Desayuno comida_cena
9,Zumo de tomate Bonnysa,"Botella 600 ml | 3,317 €/L",1.99,comida_cena
...,...,...,...,...
5599,Albahaca Hacendado,"Bote 20 g | 4,75 €/100 g",0.95,comida_cena
5602,Perejil Hacendado,"Bote 14 g | 5,358 €/100 g",0.75,comida_cena
5605,Preparado para salmón ahumado Hacendado,"Bote 750 g | 4,40 €/kg",3.30,comida_cena
5608,Sal de ajo Hacendado,"Bote 130 g | 10,385 €/kg",1.35,comida_cena


In [40]:
for key, valjue in disccionario_2_1.items():
    print(key)
    df_ = df1[df1['tipo'].str.contains(key)]
    df_.to_excel(key+'.xlsx', index=False)

Desayuno
comida_cena
entre_horas
Hogar
Cuidado_personal


###
- cesta 
- agrupar productos
- clustering de clientes

- 3 cestas: gourmet (primeras marcas, mas caros...), clasica (lo que pide siempre, (leche)), ofertas (contenga )